The goal of this exploration is to try to identify interaction terms that might be significant predictors.

For example, we already know that reference ranges for various BIA measures are differentiated by sex and age, so it seems reasonable that we might find interactions.

This work is slightly informed by my reading an online textbook about feature engineering: https://bookdown.org/max/FES/

This notebook will use a couple of different techniques to explore potential interactions. Some will be pretty naive, and others somewhat uninformed (i.e., informed by internet forums)

Planned approaches:
1. Use a random forest regressor (as suggested in this stackexchange forum: https://stats.stackexchange.com/questions/4901/what-are-best-practices-in-identifying-interaction-effects)
2. Including interaction terms in a model and then using Lasso regression (as suggested in the same forum, and also in the textbook listed above). See https://www.geeksforgeeks.org/feature-selection-using-selectfrommodel-and-lassocv-in-scikit-learn/ for instructions
3. Starting with a full model and testing interaction terms one-by-one. This has the issue of generating false positives. Potential solutions are a Bonferroni correction (likely too conservative) and a False Discovery Rate correction (which I still need to learn more about). Or maybe split the data in half, use half for testing and then run a more limited number of planned contrasts on the saved data.
4. Comparing each pairwise model with a model that also includes the interaction term. This has the same issues/limiations as the previous approach

All of these approaches need to be conducted using cross-validation in some way to try to reduce Type 1 errors.

I'm also thinking that I should start with some EDA to identify variables that are highly correlated and, probably, only use one of each set of highly-correlated variables in cases where I'm using a full model.

In [2]:
import pandas as pd
import numpy as np

Start by loading the data and merging it with the accelerometer data generated by Accelerometer_enmo_anglez_daily_averages.

In [22]:
# Load the data set train_imp.csv
train = pd.read_csv('train_cleaned.csv')

# Load the accelerometer data set Accelerometer_enmo_anglez_daily_averages.csv
accel = pd.read_csv('Accelerometer_enmo_anglez_daily_averages.csv')

# Merge train  on the 'id' column and accel on the 'ID' column
train_merged = train.merge(accel, left_on='id', right_on='ID')

# Create a data frame from accel that only includes ID values for which there is no match in the id column of train
accel_missing = accel[~accel['ID'].isin(train['id'])]

# It seems unlikly that we're going to want the ENMO_Avg_All_Days_MVPA192 or ENMO_Avg_All_Days_MVPA110 or Positive_Anglez_All_Days variables, so remove them
train_merged = train_merged.drop(columns=['ENMO_Avg_All_Days_MVPA192', 'ENMO_Avg_All_Days_MVPA110', 'Positive_Anglez_All_Days'])

In [23]:
accel_missing.head(10)

,ID,ENMO_Avg_Active_Days_MVPA192,ENMO_Avg_Active_Days_MVPA110,ENMO_Avg_All_Days_MVPA192,ENMO_Avg_All_Days_MVPA110,Positive_Anglez_Active_Days,Positive_Anglez_All_Days
4,19455336,14.000000,36.250000,8.777778,18.166667,151.500000,75.733333
5,ca33a5e7,0.000000,0.000000,3.000000,5.666667,73.333333,32.230769
8,6b6467f4,0.000000,0.000000,5.000000,13.000000,0.000000,10.000000
22,b447e66d,14.818182,38.000000,14.869565,37.739130,136.772727,126.375000
24,adbd6839,18.400000,42.800000,17.761905,41.523810,41.200000,37.954545
38,035c96dd,4.000000,13.250000,4.000000,9.166667,63.000000,50.000000
40,ab16a20d,13.800000,37.000000,11.545455,29.545455,37.400000,28.583333
43,070386b2,7.500000,20.714286,7.307692,20.600000,68.812500,64.000000
45,3f1f23e7,19.428571,48.133333,17.210526,40.000000,88.133333,75.050000
46,e46417a7,12.842105,36.263158,12.280000,26.322581,60.000000,35.048780


There are lots of NaN values. We'll be imputing some/all of these later, but it's problematic to impute prior to doing our feature engineering. So let's explore the NaN distribution for the variables

In [21]:
# Compute the number of rows in train
print('There are',train.shape[0],'rows in train')
print('There are',accel.shape[0],'rows in accel')
print('There are',train_merged.shape[0],'rows in train merged')

# Compute the number of NaN values for each variable in train. Make a dataframe of the variables and their NaN counts
nan_counts = pd.DataFrame(train_merged.isnull().sum(), columns=['nan_count'])

# Identify the five variables with the largest values in nan_count
nan_counts = nan_counts.sort_values(by='nan_count', ascending=False)
nan_counts.head()

There are 3168 rows in train
There are 996 rows in accel
There are 796 rows in train merged


,nan_count
Physical-Waist_Circumference,757
PAQ_A-PAQ_A_Total,688
PAQ_A-Season,688
Fitness_Endurance-Time_Mins,559
Fitness_Endurance-Max_Stage,559


In [24]:
nan_counts.head(20)

,nan_count
Physical-Waist_Circumference,757
PAQ_A-PAQ_A_Total,688
PAQ_A-Season,688
Fitness_Endurance-Time_Mins,559
Fitness_Endurance-Max_Stage,559
Fitness_Endurance-Time_Sec,559
FGC-FGC_GSD_Zone,545
FGC-FGC_GSND_Zone,545
FGC-FGC_GSND,542
FGC-FGC_GSD,542


**RF Regressor Interaction Terms**
(from Brave AI)

Random Forest Regressor is a powerful ensemble learning algorithm that can capture complex relationships between variables, including interaction terms. However, it does not explicitly model interactions like linear regression does with polynomial terms. Instead, Random Forest Regressor identifies interactions through feature selection and variable importance.

Approach

Prepare your data: Ensure your dataset is clean, and features are scaled (e.g., using StandardScaler from scikit-learn).

Split your data: Divide your dataset into training (e.g., 80%) and testing sets (e.g., 20%) using train_test_split from scikit-learn.

Train a Random Forest Regressor: Use RandomForestRegressor from scikit-learn with default hyperparameters (e.g., n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1).

Compute feature importance: Use the feature_importances_ attribute of the trained Random Forest Regressor to identify the most important features.

Inspect feature interactions: Analyze the feature importance scores and look for features that have high importance and are correlated with each other. This can indicate potential interaction terms.

The code example below identifies the most important features using Random Forest Regressor’s feature importance scores. It then computes the correlation matrix between these important features and looks for pairs with a correlation coefficient greater than 0.5. This can indicate potential interaction terms between the features.

Limitations
While Random Forest Regressor can identify interactions, it may not capture all possible interactions, especially those with non-linear relationships. Additionally, the feature importance scores may not always accurately reflect the strength of interactions.

In [ ]:
# Here is code to do random forest resgressor (thanks, Brave AI!)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset
df = pd.read_csv('your_data.csv')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_scaled, y_train)

# Compute feature importance
importances = rf.feature_importances_

# Inspect feature interactions
correlation_matrix = X_train_scaled.corr()
important_features = [feature for feature, importance in zip(X_train.columns, importances) if importance > 0.1]
for feature1, feature2 in itertools.combinations(important_features, 2):
    if abs(correlation_matrix[feature1][feature2]) > 0.5:
        print(f"Potential interaction: {feature1} and {feature2}")

Next we'll try using LASSO on a full model including interaction terms

Some ideas here: https://www.geeksforgeeks.org/feature-selection-using-selectfrommodel-and-lassocv-in-scikit-learn/

In [ ]:
# Import libraries
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LassoCV 
from sklearn.feature_selection import SelectFromModel 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
from sklearn.datasets import load_breast_cancer 
from sklearn.ensemble import RandomForestClassifier 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Load the Breast Cancer dataset 
cancer = load_breast_cancer() 
X, y = cancer.data, cancer.target 
  
# Split the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit LassoCV model 
lasso_cv = LassoCV(cv=5) 
lasso_cv.fit(X_train, y_train) 

# Feature selection 
sfm = SelectFromModel(lasso_cv, prefit=True) 
X_train_selected = sfm.transform(X_train) 
X_test_selected = sfm.transform(X_test) 

# Train a Random Forest Classifier using the selected features 
model = RandomForestClassifier(n_estimators=100, random_state=42) 
model.fit(X_train_selected, y_train) 


# Evaluate the model 
y_pred = model.predict(X_test_selected) 
print(classification_report(y_test, y_pred)) 

# Analyze selected features and their importance 
selected_feature_indices = np.where(sfm.get_support())[0] 
selected_features = cancer.feature_names[selected_feature_indices] 
coefficients = lasso_cv.coef_ 
print("Selected Features:", selected_features) 
print("Feature Coefficients:", coefficients) 

# Extract the selected features from the original dataset 
X_selected_features = X_train[:, selected_feature_indices] 

# Create a DataFrame for better visualization 
selected_features_df = pd.DataFrame(X_selected_features, columns=selected_features) 

# Add the target variable for coloring 
selected_features_df['target'] = y_train 

# Plot the two most important features 
sns.scatterplot(x='mean area', y='worst area', hue='target', data=selected_features_df, palette='viridis') 
plt.xlabel('Mean Area') 
plt.ylabel('Worst Area') 
plt.title('Scatter Plot of Two Most Important Features - Breast Cancer Dataset') 
plt.show() 


Below is my old code for trying to do Ridge CV. Keeping it here in case it's helpful

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Set up the kfold split
num_splits = 5
kfold = KFold(num_splits, shuffle=True)

# Define a range of alpha values
alphas = [0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]
numalphas = len(alphas)

# Create an empty array with num_splits rows and numalphas columns
rmses = np.zeros((num_splits, numalphas))

# Iterate over the three data sets
listofdatasets = [train_physical, train_fitness, train_bia]

# A data frame to store the optimal alpha values
bestalphas = pd.DataFrame(index=range(0,len(listofdatasets)))
bestalphas['dfname'] = ''
bestalphas['best_alpha_manual'] = np.nan

k=0
for df in listofdatasets:
    i = 0
    for train_index, test_index in kfold.split(df):
        tt_X = df.iloc[train_index].drop(columns=['PCIAT-PCIAT_Total'])
        tt_y = df.iloc[train_index]['PCIAT-PCIAT_Total']
        ho_X = df.iloc[test_index].drop(columns=['PCIAT-PCIAT_Total'])
        ho_y = df.iloc[test_index]['PCIAT-PCIAT_Total']

        # Iterate over alpha values with counter j
        j = 0
        for alpha in alphas:
            ridge_pipe = Pipeline([('scale', StandardScaler()),('ridge', Ridge(alpha=alpha, max_iter=5000000) )])
            ridge_pipe.fit(tt_X, tt_y)
            y_pred = ridge_pipe.predict(ho_X)
            rmses[i, j] = root_mean_squared_error(ho_y, y_pred)
            
            j=j+1

        i=i+1

    # Compute the mean of each column of rmses
    mean_rmses_within_alphas = np.mean(rmses, axis=0)

    # Compute the mean and standard deviation of each row of rmses
    mean_rmses = np.mean(mean_rmses_within_alphas, axis=0)
    std_rmses = np.std(mean_rmses_within_alphas, axis=0)

    # Identify the column of min_rmse that contains the minimum value
    best_alpha_index = np.argmin(mean_rmses_within_alphas)

    bestalphas.loc[k,'dfname'] = df.name
    bestalphas.loc[k,'best_alpha_manual'] = alphas[best_alpha_index]

    print('The alpha value with the lowest RMSE for the', df.name ,'variables is', alphas[best_alpha_index],'. The mean RMSE was', mean_rmses, ' and the standard deviation was', std_rmses )
    i=0
    j=0
    k=k+1

The alpha value with the lowest RMSE for the train_physical variables is 1 . The mean RMSE was 18.671645670338425  and the standard deviation was 0.06737420892180142
The alpha value with the lowest RMSE for the train_fitness variables is 10 . The mean RMSE was 19.05639718656873  and the standard deviation was 0.1264963289808006
The alpha value with the lowest RMSE for the train_bia variables is 10 . The mean RMSE was 18.689746957787783  and the standard deviation was 0.04291388407312609


In [41]:
bestalphas

,dfname,best_alpha_manual
0,train_physical,0.1
1,train_fitness,10.0
2,train_bia,10.0


Just for kicks, I'm going to try this out with the RidgeCV class

This will make it easier for me to try a wider range of alpha values

In [ ]:
from sklearn.linear_model import LassoCV

# A data frame to store the optimal alpha values
bestalphas = pd.DataFrame(index=range(0,len(listofdatasets)))
bestalphas['dfname'] = ''
bestalphas['best_alpha_manual'] = np.nan
bestalphas['best_alpha_automatic'] = np.nan

alphas = 10**np.linspace(10,-2,100)*0.5
#alphas = [0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]

for df in listofdatasets:
    X_train = df.drop(columns=['PCIAT-PCIAT_Total'])
    y_train = df['PCIAT-PCIAT_Total']
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_std = scaler.transform(X_train)
    lassocv = LassoCV(alphas = alphas, scoring = 'neg_root_mean_squared_error')
    lassocv.fit(X_std, y_train)
    bestalphas.loc[bestalphas['dfname']==df.name,'best_alpha_automatic']=lassocv.alpha_.astype(np.float64)

In [62]:
bestalphas

,dfname,best_alpha_manual,best_alpha_automatic
0,train_physical,1.0,2.320794
1,train_fitness,10.0,16.372746
2,train_bia,10.0,16.372746


Now that the hyperparameter is tuned, we'll compare the performance of the ridge regression and a PCA.

Note that in previous explorations we've identified n=3 as the "ideal" number of PCA components for each set of predictor variables

In [11]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression


for df in listofdatasets:
    # Identify the best alpha value we computed earlier
    best_alpha = bestalphas.loc[bestalphas['dfname'] == df.name, 'best_alpha'].values[0]
    
    # Instantiate some models. From previous exploration, we've been using 3 components for the PCA
    ridge_pipe = Pipeline([('scale', StandardScaler()), ('ridge', Ridge(alpha = best_alpha, max_iter=5000000))])
    pca_pipe = Pipeline([('scale', StandardScaler()), ('pca', PCA(n_components=3)), ('reg', LinearRegression())])

    # The training data
    X_train = df.iloc[train_index].drop(columns=['PCIAT-PCIAT_Total'])
    y_train =  df.iloc[train_index]['PCIAT-PCIAT_Total']

    # Fit the models to the training data
    ridge_pipe.fit(X_train, y_train)
    pca_pipe.fit(X_train, y_train)

    # Find the model predictions on the training set
    ridge_train_preds = ridge_pipe.predict(X_train)
    pca_train_preds = pca_pipe.predict(X_train)

    # Find the mse on the training set
    ridge_train_rmse = root_mean_squared_error(y_train, ridge_train_preds)
    pca_train_rmse = root_mean_squared_error(y_train, pca_train_preds)

    # Results
    print(df.name, f"Ridge Training MSE: {ridge_train_rmse}")
    print(df.name, f"PCA Training MSE: {pca_train_rmse}")

train_physical Ridge Training MSE: 18.533899980839752
train_physical PCA Training MSE: 19.235425201158005
train_fitness Ridge Training MSE: 18.695066004261484
train_fitness PCA Training MSE: 19.37497603121806
train_bia Ridge Training MSE: 18.770987125663712
train_bia PCA Training MSE: 19.057632568137585
